In [1]:
import pandas as pd
from stage import fullPrint, getStage
from datetime import datetime, timedelta
import yahoo_fin.stock_info as yf
import numpy as np
import pickle
import swifter
import os

PEAK = 13
allList = []
nasdaqList = pd.read_pickle("stockData/tickerList.pkl")
allList = (nasdaqList)

SectorDict = {
    "Energy":"XLE",
    "Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Defensive":"XLP",
    "Healthcare":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrials":"XLI",
    "Utilities":"XLU",
    "Basic Materials":"XLB",
    "Financial Services":"XLF",
    "Real Estate":"XLRE"}
spList = yf.tickers_sp500()
spList = set(spList)
dictionary = {}
i = 0
allLen = str(len(nasdaqList))
stockDataTickerList = []
now = datetime.now()

In [2]:
for ticker in nasdaqList:
    try:
        df = yf.get_data(ticker, start_date="1995-01-06",end_date = now, index_as_date = True)
        df = df[~df.index.duplicated()]
        dfBuy = df
        df['Dates'] = pd.to_datetime(df.index)
        df = df[df['Dates'].dt.weekday == 4]
        df = df.drop('Dates', axis = 1)
        df.to_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
        industry = yf.get_company_info(ticker).at['sector','Value']
        dictionary[ticker] = SectorDict[industry]
        dfBuy['Dates'] = pd.to_datetime(dfBuy.index)
        dfBuy = dfBuy[dfBuy['Dates'].dt.weekday == 0]
        dfBuy = dfBuy.drop('Dates', axis = 1)
        dfBuy.to_pickle('stockData/nyseNasdaq/'+ticker+'Buy.pkl')
        stockDataTickerList.append(ticker)
    except:
        print(ticker+" not found")
    i = i+1
    print(str(i)+'/'+allLen)
try:
    sectorDict = open('stockData/nasdaq.pkl', 'wb')
    pickle.dump(dictionary, sectorDict)
    sectorDict.close()
except:
    print("Something went wrong")

1/4096
2/4096
3/4096
4/4096
5/4096
6/4096
7/4096
8/4096
9/4096
10/4096
11/4096
12/4096
13/4096
14/4096
15/4096
16/4096
17/4096
18/4096
19/4096
20/4096
21/4096
22/4096
23/4096
24/4096
25/4096
26/4096
27/4096
28/4096
29/4096
30/4096
31/4096
32/4096
33/4096
34/4096
35/4096
36/4096
37/4096
38/4096
39/4096
40/4096
41/4096
42/4096
43/4096
44/4096
45/4096
46/4096
47/4096
48/4096
49/4096
50/4096
51/4096
52/4096
53/4096
54/4096
55/4096
56/4096
57/4096
58/4096
59/4096
60/4096
61/4096
62/4096
63/4096
64/4096
65/4096
66/4096
67/4096
68/4096
69/4096
70/4096
71/4096
72/4096
73/4096
74/4096
75/4096
76/4096
77/4096
78/4096
79/4096
80/4096
81/4096
82/4096
83/4096
84/4096
85/4096
86/4096
87/4096
88/4096
89/4096
90/4096
91/4096
92/4096
93/4096
94/4096
95/4096
96/4096
97/4096
98/4096
99/4096
100/4096
101/4096
102/4096
103/4096
104/4096
105/4096
106/4096
107/4096
108/4096
109/4096
110/4096
111/4096
112/4096
113/4096
114/4096
115/4096
116/4096
117/4096
118/4096
119/4096
120/4096
121/4096
122/4096
123/4096
1

In [3]:
weights = np.arange(1,31)
sumWeights = np.sum(weights)
spList = yf.tickers_sp500()
def calculateSlopeOfTicker(df,index):
    if index < 31:
        return 0
    else:
        return df.iat[index,4]/df.iat[index-1,4]

def calculateSpyDfSlopeOfTicker(df,index):
    if index < 31:
        return 0
    else:
        return df.iat[index,4]/df.iat[index-1,8]

def product(df,index):
    if index < 30:
        return 0
    else:
        return df.iat[index,0]/df.iat[index-30,0]
def calculatePeak(df,index, diff, col):
    if index == 0:
        return df.iat[index,0]
    if index < diff:
        return max(df.iat[index-1,col],df.iat[index,0])


In [4]:
spDf = yf.get_data('^GSPC', start_date="1995-01-06",end_date = now, index_as_date = True)
spDf = spDf[~spDf.index.duplicated()]
spDf['close'] = spDf['close'].rolling(window=5).apply(lambda x: np.average(x))
spDf['Dates'] = pd.to_datetime(spDf.index)
spDf = spDf[spDf['Dates'].dt.weekday == 4]
spDf = spDf.drop('Dates', axis = 1)
spDf['percent'] = spDf.swifter.apply(lambda x: product(spDf,spDf.index.get_loc(x.name)),axis=1)
spDf['WMA30'] = spDf['close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
spDf['WMA30Slope'] = spDf.swifter.apply(lambda x: calculateSpyDfSlopeOfTicker(spDf,spDf.index.get_loc(x.name)),axis=1)
spDf.to_pickle("stockData/Spy.pkl")

Pandas Apply: 100%|██████████| 1374/1374 [00:00<00:00, 23404.46it/s]


In [5]:
for ticker in stockDataTickerList:
    try:
        stockDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
        buyDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'Buy.pkl')
        if stockDf['close'].isnull().values.any() or buyDf['open'].isnull().values.any():
            os.remove('stockData/nyseNasdaq/'+ticker+'.pkl')
            os.remove('stockData/nyseNasdaq/'+ticker+'Buy.pkl')
        stockDf['fiveYearHigh'] = stockDf['close'].rolling(window=260).max().shift(1).fillna(0)  
        stockDf['WMA30'] = stockDf['close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
        stockDf['WMA30Slope'] = stockDf.swifter.apply(lambda x: calculateSlopeOfTicker(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        stockDf['volumePerc'] = stockDf['volume'].pct_change()
        stockDf['percent'] = stockDf.swifter.apply(lambda x: product(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        stockDf['RS'] = stockDf['percent'] - spDf['percent']
        #stockDf = stockDf.dropna()
        stockDf['peak'] = 0
        stockDf['trough'] = 0
        stockDf['support'] = 0
        stockDf['initialSupport'] = 0
        for index, element in stockDf.iterrows():
            numIndex =stockDf.index.get_loc(index)
            if numIndex < 260:
                stockDf.iat[numIndex, PEAK] = calculatePeak(stockDf, numIndex, 260, 3)
        stockDf['Stage'] = ""
        stockDf['secondBuy'] = False
        stockDf['resistance'] = 0
        stockDf['shortPeak'] = 0
        stockDf['shortTrough'] = 0
        print(ticker)
        stockDf.to_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
    except:
        pass


Pandas Apply: 100%|██████████| 714/714 [00:00<00:00, 22863.52it/s]


AACG


Pandas Apply: 100%|██████████| 21/21 [00:00<00:00, 11443.47it/s]


AACI


Pandas Apply: 100%|██████████| 34/34 [00:00<00:00, 25433.63it/s]


...

SCHN


Pandas Apply: 100%|██████████| 59/59 [00:00<00:00, 20716.95it/s]


SCOA


Pandas Apply: 100%|██████████| 400/400 [00:00<00:00, 17565.37it/s]


SCYX


In [6]:
transactionTemplate = yf.get_data('AAPL', start_date="1995-01-02",end_date= now, index_as_date = True, interval="1wk").drop(['open','high','low','close','adjclose','volume','ticker'],axis=1)
transactionTemplate = transactionTemplate[~transactionTemplate.index.duplicated()]
transactionTemplate.to_pickle("transactionTemplate.pkl")

In [7]:
for ticker in nasdaqList:
    try:
        stockDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
        stockDf = stockDf.drop(['open','high','low','adjclose'],axis=1)
        stockDf.to_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
    except:
        pass

In [8]:
now = datetime.now()
monday = now - timedelta(days = now.weekday())
now = now.strftime('%Y-%m-%d')
df = yf.get_data(ticker, start_date="1995-01-06",end_date = now, index_as_date = True)
df['Dates'] = pd.to_datetime(df.index)
df = df[df['Dates'].dt.weekday == 4]
df = df.drop('Dates', axis = 1)
df

,open,high,low,close,adjclose,volume,ticker
2014-05-02,97.500000,98.400002,86.300003,90.000000,90.000000,85640,SCYX
2014-05-09,89.199997,93.000000,87.000000,93.000000,93.000000,6030,SCYX
2014-05-16,90.300003,90.300003,89.699997,90.000000,90.000000,1190,SCYX
2014-05-23,89.500000,94.599998,89.500000,90.099998,90.099998,1120,SCYX
2014-05-30,84.000000,85.000000,82.500000,84.000000,84.000000,7000,SCYX
...,...,...,...,...,...,...,...
2022-03-11,4.070000,4.110000,3.890000,3.970000,3.970000,100100,SCYX
2022-03-18,4.800000,5.140000,4.760000,5.050000,5.050000,351900,SCYX
2022-03-25,4.860000,4.901000,4.630000,4.820000,4.820000,165800,SCYX
2022-04-01,3.960000,3.960000,3.660000,3.870000,3.870000,389900,SCYX


In [9]:
transactionTemplate = yf.get_data('AAPL', start_date="1995-01-06",end_date= now, index_as_date = True).drop(['open','high','low','close','adjclose','volume','ticker'],axis=1)
transactionTemplate['Dates'] = pd.to_datetime(transactionTemplate.index)
transactionTemplate = transactionTemplate[transactionTemplate['Dates'].dt.weekday == 4]
transactionTemplate = transactionTemplate.drop('Dates', axis = 1)
transactionTemplate.to_pickle("transactionTemplate.pkl")

In [2]:
SectorDict = {
    "Energy":"XLE","Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Staples":"XLP",
    "Health Care":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrial":"XLI",
    "Utilities":"XLU",
    "Materials":"XLB",
    "Financial":"XLF",
    "Real Estate":"XLRE"}
sectorDfList = []
for sector in SectorDict.values():
    sectorDfList.append(pd.read_pickle("stockData/sectorCharts/"+sector+".pkl"))
goodSectorDf = sectorDfList[0].drop(['open','high','low','adjclose','close','ticker','volume','percent','MA30','MA30Slope','RS'],axis=1)
goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
goodSectorDf.index = sectorDfList[0].index
index = 0
for date in sectorDfList[0].index:
    listOfSector = []
    for df in sectorDfList:
        try:
            if  df.at[date,'MA30Slope'] > 0.992549883242513 and df.loc[date,'RS'] > -0.3044455098712273:
                if df.at[date,'close'] > df.at[date,'MA30']:
                    #print(df.at[date,'ticker'])
                    listOfSector.append(df.at[date,'ticker'])
        except: 
            pass
        # print(listOfSector)
    goodSectorDf.iat[index,0] = set(listOfSector)
    index = index + 1


In [3]:
removeList = []
CLOSE = 0
TICKER = 2
for ticker in nasdaqList:
    try:
        df = pd.read_pickle("stockData/nyseNasdaq/"+ticker+".pkl")
        lastClose = df.iat[0,CLOSE]
        isFirst = True
        index = 0
        for date in df.Stage:
            if isFirst == True:
                isFirst = False
                continue
            if (df.iat[index,CLOSE] >= lastClose * 5):
                removeList.append(ticker)
                break
            lastClose = df.iat[index,CLOSE]
            index += 1
        stockDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
        buyDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'Buy.pkl')
        if stockDf['close'].isnull().values.any() or buyDf['open'].isnull().values.any():
            print(ticker)
            removeList.append(ticker)
            os.remove('stockData/nyseNasdaq/'+ticker+'.pkl')
            os.remove('stockData/nyseNasdaq/'+ticker+'Buy.pkl')
        df = getStage(ticker,[0.03361077383204303, 1.7466372328111242, -0.08498841770875612, 0.926192276854965, 1.1764676012556523, 1.1405501342698643, 0.8517616225271734, 0.8691969716399011, 1.268490629719207, 1.0013386634337298, 0.07551020458494859, 0.9489349345040117, 0.996709764664632, 0.9485725801058837, 0.9857019510456804],goodSectorDf)
        if df.empty:
            removeList.append(ticker)
    except:
        removeList.append(ticker)
print(removeList)

['ADTHW', 'AEAEW', 'AGRIW', 'ALACW', 'ALORW', 'APCXW', 'APR', 'ARBEW', 'ARKOW', 'BHACW', 'CBRGW', 'CELUW', 'CMPOW', 'CRECW', 'DAVEW', 'DISCB', 'DWACW', 'ENERW', 'FATPW', 'FNVTW', 'FOUNW', 'GLLIW', 'GSKY', 'GSMGW', 'HLXA', 'HMHC', 'HYMCW', 'INTEW', 'ISBC', 'JMACW', 'LEVL', 'LEVLP', 'LGSTW', 'LGTOW', 'LIONW', 'MAAQW', 'NRXPW', 'OTMOW', 'OXBRW', 'PCXCW', 'PIIIW', 'PPGH', 'PPGHU', 'RESN', 'RNERW', 'ROSEW', 'SBFMW', 'SLDPW', 'SQFTW', 'TLMD', 'VIEWW', 'VORBW', 'VWEWW', 'WEJOW', 'XFINW', 'LIONW']


In [4]:
for ticker in removeList:
    nasdaqList.remove(ticker)
with open('stockData/tickerList.pkl', 'wb') as f:
    pickle.dump(nasdaqList, f)